In [1]:
from data.fetch_data import *
from strategies.alphas import *
from utils.ga_utils import *
import warnings
from utils.optimizer import *
from utils.data_utils import * 

warnings.filterwarnings("ignore")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_data = fetch_stock_data("^NSEI",'2022-08-01','2023-08-08')
test_data= yf.download("^NSEI", start = '2023-08-01',end='2024-02-06')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
def evaluate_strategy(data,alpha_func,*args):
    data["alpha"] = alpha_func(data,*args[:-3])
    data["alpha"] = normalize_alphas(data["alpha"])
    threshold = args[-2]
    # threshold_exit =args[-2]

    stop_loss_percentage =args[-1]
    data['MACD'], data['Signal'] = calculate_macd(data, 2, 3)

    data = data.dropna()

    position = 0  # 0 for no position, 1 for long position
    profit = 0
    total_trades = 0
    winning_trades = 0
    stricly_winning_trades = 0
    allocated_funds = 100000
    no_of_shares = 0
    stop_loss_price = 0
    returns = []

    for i in range(1,len(data)):
        try:

            if position == 0:
                if data['alpha'][i] > threshold: 
                
                    position = 1  # Buy (enter long position)
                    entry_price = data['Close'][i]
                    stop_loss_price = entry_price - (entry_price * 0.01 * stop_loss_percentage)
                    # print(f"Entry: {entry_price} on {data.index[i]} Stop Loss: {stop_loss_price}")
                    total_trades += 1
                    no_of_shares = allocated_funds//entry_price
                    

                    
            elif position == 1:# and data['alpha'][i] < threshold:
                
                
                if data['Close'][i] > entry_price:  # Adjust stop loss only when the price goes up
                    stop_loss_price = max(stop_loss_price, data['Close'][i] - (data['Close'][i] * 0.01 * stop_loss_percentage))
                    
                if data['Low'][i] < stop_loss_price :#:or data['alpha'][i] < threshold_exit:  # add condition for time based exit here
                    position = 0  # Sell due to stop loss
                    exit_price = data['Open'][i] if stop_loss_price > data['Open'][i] else stop_loss_price
                    no_of_shares = allocated_funds//entry_price
                    trade_profit = (exit_price - entry_price) * no_of_shares  # Calculate profit/loss
                    profit=trade_profit
                    allocated_funds += profit
                    return_on_trade = (exit_price -entry_price) * 100/entry_price
                    returns.append(return_on_trade)
                    # print(f"Exit: {exit_price} on {data.index[i]} exited on stop loss Allocated Funds {allocated_funds}")
                    # print(f"Profit: {profit}")
                    
                    # mask = (results_data['Symbol'] == symbol) & (results_data['Exit_Date'].isnull())
                    # results_data.loc[mask, 'Exit_Date'] = data.index[i]
                    # results_data.loc[mask, 'Exit_Price'] = exit_price
           
                    if(exit_price>= entry_price):
                        winning_trades += 1
                    if(exit_price> entry_price):
                        stricly_winning_trades += 1
                # elif data['MACD_E'][i] < data['Signal_E'][i]:
                #     position = 0  # Sell due to exit condition
                #     exit_price = data['Close'][i]
                #     trade_profit = (exit_price - entry_price) * no_of_shares  # Calculate profit/loss
                #     profit=trade_profit#                     
                #     # print(f"Exit: {exit_price} on {data.index[i]} exited on stop loss Allocated Funds {allocated_funds}")
                #     # print(f"Profit: {profit}")

                #     allocated_funds += profit
                    
                    
                #     # mask = (results_data['Symbol'] == symbol) & (results_data['Exit_Date'].isnull())
                #     # results_data.loc[mask, 'Exit_Date'] = data.index[i]
                #     # results_data.loc[mask, 'Exit_Price'] = exit_price

                #     no_of_shares =0
                #     if(exit_price>= entry_price):
                #         winning_trades += 1
                #     if(exit_price> entry_price):
                #         stricly_winning_trades += 1
            
        except Exception as ex:
            print("Something caused the error",ex)
            break
        win_rate = winning_trades/total_trades if total_trades != 0 else 0

            # Calculate metrics
        metrics = {
            "Profit": profit,
            "Total Trades": total_trades,
            "Winning Trades": winning_trades,
            "Strictly Winning Trades": stricly_winning_trades,
            "Win Rate": win_rate,
            "Allocated Funds": allocated_funds,
            "Sharpe Ratio": calculate_sortino_ratio(pd.Series(returns))
        }

    return metrics
    


In [9]:
met = evaluate_strategy(test_data,alpha102,3, 8,0.55,0.9)
met

{'Profit': 409.6015625,
 'Total Trades': 7,
 'Winning Trades': 4,
 'Strictly Winning Trades': 4,
 'Win Rate': 0.5714285714285714,
 'Allocated Funds': 104169.33492968752,
 'Sharpe Ratio': 2.6462944460959634}

In [10]:
met

{'Profit': 409.6015625,
 'Total Trades': 7,
 'Winning Trades': 4,
 'Strictly Winning Trades': 4,
 'Win Rate': 0.5714285714285714,
 'Allocated Funds': 104169.33492968752,
 'Sharpe Ratio': 2.6462944460959634}

In [ ]:
met

In [6]:
# run_ga_optimization(stock_data,generate_individual,alpha8,evaluate_strategy)

In [ ]:
results_dict ={}

In [11]:
import importlib
tickers = []
alpha_function_names = ["alpha"+str(x) for x in range(1,13)]
alpha_functions = {}

for alpha_name in alpha_function_names:
    try:
        print(f"for {alpha_name}\n")
        module = importlib.import_module("strategies.alphas")
        alpha_function = getattr(module, alpha_name)
        alpha_functions[alpha_name] = alpha_function

        best_metrics, best_individual = run_ga_optimization(train_data, generate_individual, alpha_function, evaluate_strategy)

        results_dict[alpha_function.__name__] = {
        # "alpha_name": alpha_function.__name__,
        "best_metrics": best_metrics,
        "best_individual": best_individual,
        "ticker": "BPCL.NS"
    }
        
    except :
        print(f"Failed {alpha_name}")





for alpha1

[100000, 100000, 100000]
Saturation condition met, breaking the loop

Best Strategy Metrics (Generation Generation 1):

Best Individual Parameters: [3, 8, 7, 0.31491501609895356, 0.6166850217632279]
Profit: 0
Total Trades: 0
Winning Trades: 0
Strictly Winning Trades: 0
Win Rate: 0
Allocated Funds: 100000
Sharpe Ratio: nan
for alpha2



In [17]:
generate_individual(alpha2)

{'delta_lookback': 2,
 'window_corr': 8,
 'threshold': 0.013352781684141113,
 'stop_loss_percentage': 0.6000600239546062}

In [11]:
pd.DataFrame(results_dict).to_csv("result_12.csv",index = False)

In [52]:
res = pd.read_csv("result_12.csv")
res

,alpha2,alpha3,alpha4,alpha5,alpha6,alpha7,alpha8,alpha9,alpha10,alpha11,alpha12
0,"{'Profit': 51.70458984375, 'Total Trades': 115...","{'Profit': 49.50439453125, 'Total Trades': 124...","{'Profit': 48.404296875, 'Total Trades': 125, ...","{'Profit': 310.49560546875, 'Total Trades': 11...","{'Profit': 48.404296875, 'Total Trades': 124, ...","{'Profit': 52.8046875, 'Total Trades': 118, 'W...","{'Profit': 51.70458984375, 'Total Trades': 116...","{'Profit': 524.597900390625, 'Total Trades': 2...","{'Profit': 524.597900390625, 'Total Trades': 2...","{'Profit': 51.70458984375, 'Total Trades': 113...","{'Profit': 303.595703125, 'Total Trades': 125,..."
1,"[7, 14, 0, -0.9867023487400403]","[3, 0, -0.9994148320141686]","[9, 0, -0.9843327836088254]","[4, 0, -0.9794146305341901]","[4, 0, -0.998063993849938]","[4, 20, 0, -0.9970650140146375]","[10, 8, 0, -0.9983180708342115]","[7, 0, 1.963733178971668]","[4, 0, -0.6858613288110106]","[17, 9, 0, -0.9700672832258828]","[6, 0, -0.8079972587336343]"
2,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS,BPCL.NS
